In [8]:
import pandas as pd
from glob import glob

In [16]:
# INPUT_CSV_FILES = [
#     'predictions/predictions-c.csv',
#     'predictions/predictions-d2.csv',
#     'predictions/predictions-e@17.csv',
#     'predictions/predictions-f@13.csv',
#     'predictions/predictions-f3@18.csv',
#     'predictions/predictions-f2@21.csv',
# ]

INPUT_CSV_FILES = glob('predictions/*.csv')
print(INPUT_CSV_FILES)

OUT_FILE = 'submissions/s-13.csv'

ENSEMBLING_METHOD = 'mean' # mean/voting

['predictions/predictions-e@7.csv', 'predictions/predictions-f@13.csv', 'predictions/predictions-f2@21.csv', 'predictions/predictions-d2.csv', 'predictions/predictions-e@17.csv', 'predictions/predictions-c.csv', 'predictions/predictions-e@11.csv', 'predictions/predictions-f3@34.csv', 'predictions/predictions-f3@18.csv']


In [12]:
%run 'lib.ipynb'
%run 'data-generator.ipynb'

In [13]:
dfs = [pd.DataFrame.from_csv(f) for f in INPUT_CSV_FILES]

In [14]:
predictions = {} # file: [pq, p2,...]

for df in dfs:
    for file, p in df.iterrows():
        if not file in predictions:
            predictions[file] = []
        predictions[file].append(list(p))

In [15]:
submission = {}
mean_voting_disagrees = 0

for file, preds in predictions.items():

    label_i_mean = np.argmax(np.mean(preds, axis=0))
    label_i_voting = np.argmax(np.bincount(np.argmax(preds, axis=1)))
    if label_i_mean != label_i_voting: mean_voting_disagrees += 1

    if ENSEMBLING_METHOD == 'mean':
        submission[file] = LABELS[label_i_mean]
    elif ENSEMBLING_METHOD == 'voting':
        submission[file] = LABELS[label_i_voting]

print('mean_voting_disagrees', mean_voting_disagrees)

mean_voting_disagrees 4652


In [17]:
submission_df = pd.DataFrame.from_dict(submission, orient='index')
submission_df.columns = ['label']
submission_df.to_csv(OUT_FILE, index_label='fname')